# Linear Regression

In [10]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
data = load_diabetes()
X = data['data']
y = data['target']
column_names = data['feature_names']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# OLS via Augmentation & Inversion

Given a dataset with:

- Feature matrix $X \in \mathbb{R}^{N \times p}$
- Target vector $y \in \mathbb{R}^N$

We fit a linear model with weights $W$ and bias $b$:  
$$
\hat{y} = X\,W + b\,\mathbf{I}
$$

## 1. Augmented Design Matrix

We augment \(X\) by adding a column of ones:  
$$
X_{\text{aug}} = \bigl[\,X \;\big|\; \mathbf{1}\bigr] \tag{1}
$$

and define the parameter vector:  
$$
\beta = \begin{pmatrix} W \\[4pt] b \end{pmatrix} \tag{2}
$$

The model can now be written as:  
$$
\hat{y} = X_{\text{aug}}\,\beta \tag{3}
$$

## 2. Least Squares Objective

The OLS loss is  
$$
L(\beta) = \|\,y - X_{\text{aug}}\,\beta\|_2^2 \tag{4}
$$

Expanding:  
$$
\begin{aligned}
L(\beta)
&= (y - X_{\text{aug}}\beta)^\top (y - X_{\text{aug}}\beta) \\[6pt]
&= y^\top y \;-\; 2\,\beta^\top X_{\text{aug}}^\top y \;+\; \beta^\top X_{\text{aug}}^\top X_{\text{aug}}\,\beta
\end{aligned}
$$

## 3. Solve for $\beta$

Set the gradient to zero:  
$$
\begin{aligned}
\frac{\partial L}{\partial \beta}
&= -2\,X_{\text{aug}}^\top (y - X_{\text{aug}}\beta) \\[4pt]
&= -2\,X_{\text{aug}}^\top y \;+\; 2\,X_{\text{aug}}^\top X_{\text{aug}}\,\beta
\;=\;0
\end{aligned}
$$

Rearrange to get the normal equation:  
$$
X_{\text{aug}}^\top X_{\text{aug}}\,\beta \;=\; X_{\text{aug}}^\top y
$$

Hence the closed-form solution is  
$$
\boxed{
\beta 
= \bigl(X_{\text{aug}}^\top X_{\text{aug}}\bigr)^{-1}\,X_{\text{aug}}^\top\,y
} \tag{5}
$$

# Solving with Matrix Inversion

In [ ]:
# 1. Augment training features with a column of ones
X_aug = np.hstack([X_train, np.ones((len(X_train), 1))]) # X_aug = [X_train, 1] # Add a column of ones for the bias term

# 2. Apply the normal equation
beta = np.linalg.inv(X_aug.T @ X_aug) @ (X_aug.T @ y_train) # (X_aug^T X_aug)^{-1} X_aug^T y_train

# 3. Unpack weights and bias
W, b = beta[:-1], beta[-1] # W = beta[:-1] and b = beta[-1] # Last element is the bias term

y_pred = X_test @ W + b # Y = X@W + b
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)
r2 = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2))
print(f"RMSE: {rmse:.2f}, R^2: {r2:.2f}")
# Get weights and bias from matrix inversion solution


RMSE: 53.85, R^2: 0.45


# Solving with sklearn LinearRegression implementation

In [18]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)
r2 = model.score(X_test, y_test)
print(f"RMSE: {rmse:.2f}, R^2: {r2:.2f}")


RMSE: 53.85, R^2: 0.45


## PCA via Eigen-Decomposition of the Covariance (U) Matrix

1. **Center** your data:  
   $$X_c = X - \operatorname{mean}(X,\;{\rm axis}=0)$$  
2. **Form** the covariance‐style matrix:  
   $$U = \frac{1}{N-1}\,X_c^\top X_c$$  
3. **Eigendecompose** \(U\):  
   $$U\,v_i = \lambda_i\,v_i$$  
4. **Compare** $\{\lambda_i,\,v_i\}$ to `sklearn.decomposition.PCA`.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# --- assume X_train is your (N, p) feature matrix ---
N, p = X_train.shape

# 1. Center
Xc = X_train - X_train.mean(axis=0)

# 2. Covariance-style matrix
U = Xc.T @ Xc / (N - 1)

# 3. Eigendecomposition
eigvals, eigvecs = np.linalg.eig(U)
idx = np.argsort(eigvals)[::-1] # Sort eigenvalues and eigenvectors in descending order
eigvals, eigvecs = eigvals[idx], eigvecs[:, idx]

# 4. sklearn PCA
pca = PCA().fit(X_train)

# Print and compare
print("Eigenvalues (manual):   ", np.round(eigvals, 4))
print("Explained var (sklearn):", np.round(pca.explained_variance_, 4))

# Compare first two principal directions (up to sign)
for i in range(2):
    manual_pc = eigvecs[:, i]
    skl_pc    = pca.components_[i]
    corr = np.dot(manual_pc, skl_pc)
    print(f"PC{i+1} alignment:", np.sign(corr)*np.round(np.abs(corr),4))


Eigenvalues (manual):    [0.009  0.0033 0.0028 0.0023 0.0015 0.0014 0.0012 0.001  0.0002 0.    ]
Explained var (sklearn): [0.009  0.0033 0.0028 0.0023 0.0015 0.0014 0.0012 0.001  0.0002 0.    ]
PC1 alignment: 1.0
PC2 alignment: 1.0


# Additional Analysis

## Eigen‐Decomposition of $X_{\text{aug}}^T X_{\text{aug}}$ and Inversion

Given the augmented design matrix  
$$
X_{\text{aug}} \in \mathbb{R}^{N\times (p+1)},
$$
the Gram matrix  
$$
U = X_{\text{aug}}^T X_{\text{aug}}
$$
is square of shape $(p+1)\times(p+1)$.  We can diagonalize it as follows:

1. **Eigendecomposition**  
   $$
   U = V \, D \, V^T,
   $$
   where  
   - $D = \mathrm{diag}(d_1, \dots, d_{p+1})$  
   - $V^T V = I$  

2. **Inverse via eigenvalues**  
   $$
   U^{-1} = V \, D^{-1} \, V^T,
   $$
   with  
   $$
   D^{-1} = \mathrm{diag}\bigl(\tfrac1{d_1}, \dots, \tfrac1{d_{p+1}}\bigr).
   $$

3. **Solve for** $\beta$  
   Plugging into the normal equation
   $$
   \beta = U^{-1}\,X_{\text{aug}}^T\,y
   $$
   gives the closed-form
   $$
   \boxed{
     \beta = V\,D^{-1}\,V^T\,X_{\text{aug}}^T\,y
   }
   $$


In [ ]:
U = X_aug.T @ X_aug 
eigvals, eigvecs = np.linalg.eig(U)
idx = np.argsort(eigvals)[::-1]  # Sort eigenvalues and eigenvectors in descending order
eigvals, eigvecs = eigvals[idx], eigvecs[:, idx]

D = np.diag(eigvals)
V = eigvecs
VT = eigvecs.T
inv_D = np.diag(1 / eigvals) 
inv_U = V @ inv_D @ V.T 
beta_matrix_inv = inv_U @ X_aug.T @ y_train
W_matrix_inv, b_matrix_inv = beta_matrix_inv[:-1], beta_matrix_inv[-1]  # Last element is the bias term

y_pred_matrix_inv = X_test @ W_matrix_inv + b_matrix_inv  # Y = X@W + b
mse_matrix_inv = np.mean((y_test - y_pred_matrix_inv) ** 2)
rmse_matrix_inv = np.sqrt(mse_matrix_inv)
r2_matrix_inv = 1 - (np.sum((y_test - y_pred_matrix_inv) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2))
print(f"Matrix Inversion RMSE: {rmse_matrix_inv:.2f}, R^2: {r2_matrix_inv:.2f}")

Matrix Inversion RMSE: 53.85, R^2: 0.45


## Ridge Regression via Augmentation & Eigen‐Inversion

We start with the augmented design matrix  
$$
X_{\text{aug}} = \left[\, X \;\big|\; \mathbf{1} \,\right] \in \mathbb{R}^{N \times (p+1)}
$$
and solve the regularized least‐squares problem
$$
\underset{\beta}{\min}\;\|\,y - X_{\rm aug}\beta\|_2^2 \;+\;\lambda\|\beta\|_2^2.
\tag{1}
$$

### 1. Normal equation for Ridge

Taking the gradient and setting to zero gives
$$
X_{\rm aug}^T X_{\rm aug}\,\beta + \lambda\,\beta 
= X_{\rm aug}^T y
\quad\Longrightarrow\quad
\boxed{
  (X_{\rm aug}^T X_{\rm aug} + \lambda I)\,\beta
  = X_{\rm aug}^T y
}
\tag{2}
$$
so the closed‐form is
$$
\beta_{\rm ridge}
= \bigl(X_{\rm aug}^T X_{\rm aug} + \lambda I\bigr)^{-1}\,X_{\rm aug}^T y.
\tag{3}
$$

### 2. Eigen-decomposition of $X_{\text{aug}}^\top X_{\text{aug}}$


Let
$$
U = X_{\rm aug}^T X_{\rm aug}
= V\,D\,V^T,
\quad
D = \mathrm{diag}(d_1,\dots,d_{p+1}),
\quad
V^TV = I.
\tag{4}
$$

Then
$$
U + \lambda I
= V\,(D + \lambda I)\,V^T,
\quad
\bigl(U + \lambda I\bigr)^{-1}
= V\,(D + \lambda I)^{-1}\,V^T.
\tag{5}
$$

### 3. Ridge solution via eigen‐inversion

Substitute into (3):
$$
\boxed{
  \beta_{\rm ridge}
  = \bigl(U + \lambda I\bigr)^{-1}\,X_{\rm aug}^T y
  = V\,(D + \lambda I)^{-1}\,V^T\,X_{\rm aug}^T y
}
\tag{6}
$$

In [43]:
U = X_aug.T @ X_aug                                  # shape = (p+1, p+1)

# 3. Eigendecompose U
eigvals, eigvecs = np.linalg.eig(U)                  # eigvals.shape = (p+1,), eigvecs.shape = (p+1, p+1)
idx = np.argsort(eigvals)[::-1]                      # indices for sorting descending
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

# 4. Form the inverse of (D + λI) in the eigenbasis
lambda_ridge = 0.01
inv_D_plus = np.diag(1.0 / (eigvals + lambda_ridge)) # shape = (p+1, p+1)

# 5. Reconstruct (U + λI)^{-1} = V (D+λI)^{-1} V^T
V = eigvecs
inv_U_ridge = V @ inv_D_plus @ V.T                   # shape = (p+1, p+1)

# 6. Solve for beta_ridge = (U + λI)^{-1} X_aug^T y_train
beta_ridge = inv_U_ridge @ X_aug.T @ y_train         # shape = (p+1,)

# 7. Unpack weights and bias
W_ridge = beta_ridge[:-1]                            # shape = (p,)
b_ridge = beta_ridge[-1]                             # scalar

# 8. Predict on the test set
y_pred = X_test @ W_ridge + b_ridge                   # shape = (n_test,)

# 9. Evaluate performance
mse  = np.mean((y_test - y_pred)**2)
rmse = np.sqrt(mse)
r2   = 1 - np.sum((y_test - y_pred)**2) / np.sum((y_test - np.mean(y_test))**2)

print(f"Ridge Regression (eigen-inversion) → RMSE: {rmse:.2f},  R²: {r2:.2f}")


Ridge Regression (eigen-inversion) → RMSE: 53.69,  R²: 0.46


# sklearn ridge regression for comparison

In [ ]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.01)
ridge_model.fit(X_train, y_train)
y_pred_sklearn = ridge_model.predict(X_test)
mse_sklearn = np.mean((y_test - y_pred_sklearn) ** 2)
rmse_sklearn = np.sqrt(mse_sklearn)
r2_sklearn = ridge_model.score(X_test, y_test)
print(f"Ridge Regression (sklearn) → RMSE: {rmse_sklearn:.2f},  R²: {r2_sklearn:.2f}")

Ridge Regression (sklearn) → RMSE: 53.69,  R²: 0.46


### Commentary: Ridge Regression and Eigenvalue Adjustment

In ridge regression, the only change from ordinary least squares (OLS) happens **at the level of eigenvalue inversion**.

In OLS, the closed-form solution involves:
$$
\beta_{\text{OLS}} = V\,D^{-1}\,V^\top\,X_{\text{aug}}^\top y
$$
where:
- $V$ contains the eigenvectors of $X_{\text{aug}}^\top X_{\text{aug}}$
- $D = \mathrm{diag}(d_1, d_2, \dots, d_{p+1})$ contains its eigenvalues
- $D^{-1} = \mathrm{diag}\left(\frac{1}{d_1}, \frac{1}{d_2}, \dots, \frac{1}{d_{p+1}}\right)$

---

In **ridge regression**, we modify this inversion step:
$$
\beta_{\text{ridge}} = V\,(D + \lambda I)^{-1}\,V^\top\,X_{\text{aug}}^\top y
$$
This means that instead of dividing by $d_i$, we divide by $d_i + \lambda$.  
So the shrinkage factor becomes:
$$
\frac{1}{d_i + \lambda}
$$

---

### Why this improves stability

- If an eigenvalue $d_i$ is **very small** (i.e. a nearly flat direction), then $1/d_i$ becomes large → unstable OLS.
- Ridge fixes this by **adding** $\lambda$, making all eigenvalues invertible and bounded away from zero.
- This stabilizes the solution, especially when $X^T X$ is ill-conditioned or near-singular.

---

### When ridge has little effect

If the **smallest eigenvalue** \(d_{\min}\) is already large, then:
$$
\frac{1}{d_{\min} + \lambda} \approx \frac{1}{d_{\min}}
$$
So ridge shrinkage becomes negligible, and the solution is essentially the same as OLS:
$$
\lim_{\lambda \to 0} \beta_{\text{ridge}} = \beta_{\text{OLS}}
$$

In such cases, **regularization is not needed**, because the original matrix is already well-conditioned.
